In [ ]:
# default_exp days
import sys
sys.path.append('..')

In [ ]:
from seq_fastai.core import *
from seq_fastai.imports import *
from seq_fastai.torch_imports import *

/home/tako/devtools/fastai_dev/env/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [ ]:
import fastai2

In [ ]:
from fastai2.test import *
from fastai2.core import *
from fastai2.layers import *
from fastai2.data.all import *
from fastai2.optimizer import *
from fastai2.learner import *
from fastai2.metrics import *
from fastai2.text.core import *
from fastai2.text.data import *
from fastai2.text.models.core import *
from fastai2.text.models.awdlstm import *
from fastai2.callback.rnn import *
from fastai2.callback.all import *

In [ ]:
# export
class BaseDays(pd.DataFrame):
    def __init__(self, data, y_names, cat_var=None, con_var=None, cats={}, **kw):
        super(BaseDays, self).__init__(data, **kw)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            self.con_var, self.y_names = L(con_var), L(y_names)
            self.set_index("datetime", drop=False, inplace=True)

            self.cols = ["datetime"] + self.con_var
            self.cat_var = L(ifnone(cat_var, cats.keys()))
            self.cats = cats

        if cats == {}:
            for cat_var in self.cat_var:
                self.cats[cat_var] = self[cat_var].iloc[0]
        else:
            for k, v in cats.items():
                self.loc[:, k] = v
        for c in self.cols + self.y_names:
            if c not in self.columns:
                print(f"Warning {c} not in {self.columns} of {self.__class__}")

    def copy(self):
        # https://stackoverflow.com/questions/17591104/in-pandas-can-i-deeply-copy-a-dataframe-including-its-index-and-column
        #         print('copy made ',self.__class__)
        return self.__class__(
            pd.DataFrame(self.values.copy(), self.index.copy(), self.columns.copy()),
            self.y_names.copy(),
            self.cat_var.copy(),
            self.con_var.copy(),
        )


In [ ]:
bd = BaseDays(pd.DataFrame({'datetime':[pd.Timestamp('2019-01-01')+x*pd.Timedelta('1H') for x in range(72)],
                     'target':np.arange(72),
                     'quater':np.arange(72)/4}),
         'target',
         con_var='quater'
        )
test_eq(bd.cats,{})
test_eq(bd.con_var,['quater'])

_bd = bd.copy()
_bd['half'] = _bd['quater']*2
del _bd['quater']
_bd.con_var=L('half')
test_eq(bd['quater'],np.arange(72)/4)


In [ ]:
bd = BaseDays(pd.DataFrame({'datetime':[pd.Timestamp('2019-01-01')+x*pd.Timedelta('1H') for x in range(72)],
                     'target':np.arange(72),
                     'one':[1]*72}),
         'target',
         cat_var='one'
        )
test_eq(bd.cats,{'one':1})
test_eq(bd.cat_var,['one'])

bd = BaseDays(pd.DataFrame({'datetime':[pd.Timestamp('2019-01-01')+x*pd.Timedelta('1H') for x in range(72)],
                     'target':np.arange(72)}),
         'target',
         cats={'one':1}
        )
test_eq(bd.cats, {'one':1})
test_eq(bd.cat_var, ['one'])


In [ ]:
# export
class Days(BaseDays):
    def show(self, plot=True, **kwargs):

        title = f"{self.cats}  from {str(min(self.df.loc[:,'datetime']))} to {str(max(self.df.loc[:,'datetime']))} #{self.df.shape[0]}"
        print(self.df.columns)
        if plot:
            plot_days(
                self.df,
                left=self.con_var,
                right=self.y_names + ["pred"],
                title=title,
                **kwargs,
            )
        else:
            return title + "\n" + str(self.df.head(1))

    @classmethod
    def from_ndays(cls, nday):
        cats = {} if not hasattr(nday, "cats") else nday.cats
        return cls(nday.df, nday.y_names, nday.cat_var, nday.con_var, cats=cats)


In [ ]:
# export
class NotDays(BaseDays):
    @classmethod
    def from_days(cls, day):
        cats = {} if not hasattr(day, "cats") else day.cats
        return cls(day.df, day.y_names, day.cat_var, day.con_var, cats=cats)


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 00_torch_core.ipynb.
Converted Days.ipynb.
Converted v2_seq_seq-realdata.ipynb.
